# BBPSSW Protocol 

In [34]:
#imports
from qutip import *
from qutip_qip.circuit import QubitCircuit
from qutip_qip.operations import Gate, cnot
import numpy as np
from math import pi
import matplotlib.pyplot as plt
from qutip_qip.device import LinearSpinChain
from qutip_qip.noise import DecoherenceNoise
from qutip.qip.operations.gates import (rx, ry, rz, sqrtnot, snot, phasegate,
                                        x_gate, y_gate, z_gate, cy_gate,
                                        cz_gate, s_gate, t_gate, cs_gate,
                                        qasmu_gate, ct_gate, cphase, cnot,
                                        csign, berkeley, swapalpha, swap,
                                        iswap, sqrtswap, sqrtiswap, fredkin,
                                        toffoli, controlled_gate, globalphase,
                                        expand_operator, gate_sequence_product)

In [2]:
#initialisations
phi_plus = bell_state("00")
phi_minus = bell_state("01")
psi_plus = bell_state("10")
psi_minus = bell_state("11")

fidelity_val = 0.60

In [3]:
def F_to_theta(F):
    """Returns the angle theta corresponding to an input fidelity F.
    
    Keyword arguments:
    F -- fidelity between 0 and 1
    """
    if F < 0 or F > 1:
        raise Exception('Fidelity must be between 0 and 1.')
    else:
        theta = np.arcsin(2*F - 1)
    
    return theta

In [4]:
#generate Werner state for given fidelity values ranging between 0 and 1. this is done using random bilateral rotations. 
def Werner_state(F):
    if F < 0 or F > 1:
        raise Exception('Fidelity must be between 0 and 1.')
    
    state = F * psi_minus * psi_minus.dag() + (1 - F) / 3 * (phi_plus * phi_plus.dag() + phi_minus * phi_minus.dag() + psi_plus * psi_plus.dag())
    
    return state



In [6]:
control_qubits = [0,1]
target_qubits = [2,3]

In [7]:
def bilateral_rotation(circuit, qubits, rot_idx):
    """Adds a bilateral rotation to a quantum circuit.
    
    Keyword arguments:
    circuit --  quantum circuit to which the rotations are added
    qubits -- alist of qubits, each of which is rotated
    rot_idx -- integer that numbers the rotations from the set of all rotations, ranges from 0 to 11
    """
    for qubit_number in qubits:
        if rot_idx == 0:
            pass
        
        elif rot_idx == 1:
            circuit.add_gate("RX", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            circuit.add_gate("RX", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})

            
        elif rot_idx == 2:
            circuit.add_gate("RY", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            circuit.add_gate("RY", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            
        elif rot_idx == 3:
            circuit.add_gate("RZ", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            circuit.add_gate("RZ", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            
        elif rot_idx == 4:
            circuit.add_gate("RX", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            circuit.add_gate("RY", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            
        elif rot_idx == 5:
            circuit.add_gate("RY", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            circuit.add_gate("RZ", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            
        elif rot_idx == 6:
            circuit.add_gate("RZ", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            circuit.add_gate("RX", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})

            
        elif rot_idx == 7:
            circuit.add_gate("RY", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            circuit.add_gate("RX", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})

            
        elif rot_idx == 8:
            circuit.add_gate("RX", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            circuit.add_gate("RY", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            circuit.add_gate("RX", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            circuit.add_gate("RY", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            
        elif rot_idx == 9:
            circuit.add_gate("RY", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            circuit.add_gate("RZ", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            circuit.add_gate("RY", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            circuit.add_gate("RZ", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            
        elif rot_idx == 10:
            circuit.add_gate("RZ", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            circuit.add_gate("RX", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            circuit.add_gate("RZ", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            circuit.add_gate("RX", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})

            
        elif rot_idx == 11:
            circuit.add_gate("RY", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            circuit.add_gate("RX", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            circuit.add_gate("RY", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})
            circuit.add_gate("RX", targets=[qubit_number], arg_value=pi / 2, style={"showarg": True})

            
        else:
            raise Exception('Input argument rot_idx has to be an integer between 0 and 11.')

In [63]:
def add_qasmu_gate(circuit, theta, phi, lam, target):
    """Adds a QASMU gate to a quantum circuit.
    
    Keyword arguments:
    circuit --  quantum circuit to which the gate is added
    theta -- angle theta
    phi -- angle phi
    lam -- angle lambda
    """
    circuit.add_gate("RZ", targets=[target], arg_value=0, style={"showarg": True})
    circuit.add_gate("RY", targets=[target], arg_value=(F_to_theta(fidelity_val)), style={"showarg": True})
    circuit.add_gate("RZ", targets=[target], arg_value=0, style={"showarg": True})

In [76]:
#create circuit that generates source
qc = QubitCircuit(4)
qc.add_gate("X", targets=[1])
#qasmu gate
qc.add_gate("RZ", targets=[0], arg_value=0, style={"showarg": True})
qc.add_gate("RY", targets=[0], arg_value=(F_to_theta(fidelity_val)), style={"showarg": True})
qc.add_gate("RZ", targets=[0], arg_value=0, style={"showarg": True})

qc.add_gate("CNOT", controls=[0], targets=[1])
qc.add_gate("X", targets=[3])

#qasmu gate
qc.add_gate("RZ", targets=[2], arg_value=0, style={"showarg": True})
qc.add_gate("RY", targets=[2], arg_value=(F_to_theta(fidelity_val)), style={"showarg": True})
qc.add_gate("RZ", targets=[2], arg_value=0, style={"showarg": True})

qc.add_gate("CNOT", controls=[2], targets=[3])

#generate werner state from source using random bilateral rotations
rand_int = np.random.randint(12)
bilateral_rotation(qc, control_qubits, rand_int)
bilateral_rotation(qc, target_qubits, rand_int)
# qc.draw("matplotlib", dpi=150)

#bbpssw protocol steps  
qc.add_gate("Y", targets=[0])
qc.add_gate("Y", targets=[2])
qc.add_gate("CNOT", controls=[0], targets=[2])
qc.add_gate("CNOT", controls=[1], targets=[3])
qc.add_measurement("M0", targets=[2], classical_store=[0])
qc.add_measurement("M0", targets=[3], classical_store=[1])



In [77]:
tlist = np.linspace(0, 30., 100)
coeff = tlist * 0.01
noise = DecoherenceNoise(
    sigmam(), targets=0,
    coeff=coeff, tlist=tlist)
processor = LinearSpinChain(1)
processor.add_noise(noise)
processor.load_circuit(qc)
tlist = np.linspace(0, 20, 300)
result = processor.run_state(init_state, tlist=tlist)

NotImplementedError: Gate Measurement(M0, target=[2], classical_store=[0]) cannot be resolved.